In [27]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [28]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [29]:
subject = 'Ceará - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [30]:
data = pd.read_csv('2003_mo_model_input_CE.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t),Ceará - Consumo de Cimento (t)
0,2003-1,8.274459,0.301376,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.684040,6.965352e+07,4.525724e+06,7.532751,6.293821e+07,107.673000,81.265
1,2003-2,8.268512,0.301393,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.684476,6.974354e+07,4.530359e+06,7.535808,6.297178e+07,107.237962,65.200
2,2003-3,8.262564,0.301309,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.684912,6.983356e+07,4.534994e+06,7.538865,6.300536e+07,108.742184,55.165
3,2003-4,8.256617,0.300917,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.685347,6.992358e+07,4.539629e+06,7.541921,6.303893e+07,109.391112,53.928
4,2003-5,8.250670,0.300417,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.685783,7.001361e+07,4.544265e+06,7.544978,6.307250e+07,111.602457,65.334
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,0.535686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.122997,186.918
236,2022-9,NaN,0.533990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239.220173,179.828
237,2022-10,NaN,0.530449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,237.879214,174.940
238,2022-11,NaN,0.526253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236.799325,182.597


In [31]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t)
0,-0.777612,-1.446563,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.089980,-1.710320,-1.784591,-2.264453,-2.078282,-1.711278
1,-0.782340,-1.445945,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.056200,-1.691964,-1.739638,-2.228847,-2.046399,-1.729252
2,-0.787067,-1.449072,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.022420,-1.673607,-1.694684,-2.193241,-2.014516,-1.667105
3,-0.791795,-1.463663,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.988641,-1.655251,-1.649730,-2.157635,-1.982633,-1.640295
4,-0.796522,-1.482248,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.954861,-1.636895,-1.604776,-2.122029,-1.950750,-1.548935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.062332,1.412350,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.416090,1.084593,-1.281134,0.537648,0.738842,1.593689
188,1.063244,1.500856,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.410646,1.073027,-1.282511,0.515683,0.720806,1.645498
189,1.064157,1.599064,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.405202,1.061462,-1.283888,0.493719,0.702770,1.698639
190,1.065069,1.702017,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.399758,1.049897,-1.285265,0.471754,0.684734,1.730727


In [32]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      75.575
1      53.065
2      67.755
3      64.858
4      72.377
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Ceará - Consumo de Cimento (t), Length: 240, dtype: float64

In [33]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t)
0,-0.777612,-1.446563,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-2.089980,-1.710320,-1.784591,-2.264453,-2.078282,-1.711278
1,-0.782340,-1.445945,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-2.056200,-1.691964,-1.739638,-2.228847,-2.046399,-1.729252
2,-0.787067,-1.449072,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-2.022420,-1.673607,-1.694684,-2.193241,-2.014516,-1.667105
3,-0.791795,-1.463663,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.988641,-1.655251,-1.649730,-2.157635,-1.982633,-1.640295
4,-0.796522,-1.482248,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.954861,-1.636895,-1.604776,-2.122029,-1.950750,-1.548935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.423186,-0.021469,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.409954,1.190773,-0.838130,0.874129,1.017258,1.523635
158,1.399318,-0.031468,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.414132,1.193287,-0.881990,0.871719,1.015919,1.541381
159,1.375451,-0.044253,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.418310,1.195800,-0.925849,0.869309,1.014581,1.523536
160,1.351584,-0.046806,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.422488,1.198313,-0.969709,0.866899,1.013242,1.507028


In [34]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0       75.575
1       53.065
2       67.755
3       64.858
4       72.377
        ...   
157    134.739
158    151.819
159    124.858
160    162.684
161    169.251
Name: Ceará - Consumo de Cimento (t), Length: 162, dtype: float64

In [35]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Ceará - Desemprego,Ceará - value,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Ceará - IDH,Ceará - PIB - Estadual,Ceará - PIB - Construção Civil,Ceará - PIB - Per Capita,Ceará - PIB - Preços de Mercado,Ceará - Produção de Cimento (t)
162,1.303850,-0.008291,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,1.430845,1.203340,-1.057428,0.862078,1.010565,1.538016
163,1.279983,0.037962,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,1.435023,1.205853,-1.101287,0.859668,1.009226,1.568790
164,1.256116,0.082773,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,1.439201,1.208367,-1.145146,0.857258,1.007888,1.584409
165,1.232249,0.134604,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,1.443379,1.210880,-1.189006,0.854848,1.006549,1.606080
166,1.208382,0.172573,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,1.447557,1.213393,-1.232865,0.852438,1.005210,1.622668
167,1.184514,0.228682,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,1.451735,1.215906,-1.276725,0.850028,1.003872,1.620527
168,1.160647,0.288849,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,1.455913,1.218420,-1.320584,0.847618,1.002533,1.590480
169,1.151922,0.338531,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,1.455771,1.214014,-1.316493,0.834600,0.991080,1.556448
170,1.143197,0.385163,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,1.455628,1.209608,-1.312402,0.821582,0.979627,1.554291
171,1.134472,0.435009,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,1.455485,1.205202,-1.308311,0.808564,0.968173,1.533330


In [36]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162    179.243
163    194.134
164    186.948
165    194.907
166    179.505
167    173.278
168    160.261
169    113.987
170    114.289
171    117.313
172    110.502
173    147.996
174    152.386
175    177.259
176    155.913
177    172.410
178    179.419
179    157.165
180    151.601
181    129.537
182    105.236
183    110.835
184    127.925
185    127.138
186    157.128
187    164.440
188    140.298
189    168.407
190    163.615
191    155.070
Name: Ceará - Consumo de Cimento (t), dtype: float64

In [37]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*div_factor + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [38]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [41]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [42]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2859718792, 2989502355, 2210823185, 1253513620, 797226887, 1847975421, 2912890130, 377108361, 1805525813, 814213225, 2172303189, 1612795903, 3759746527, 4015148741, 897723027, 3326545792, 2610222674, 2778851060, 1489452228, 1422439146, 2977199175, 3564889458, 3580420273, 3490905392, 636317196]


Step: 0 ___________________________________________
val_loss: 139.54507446289062
winner_seed: 2859718792


Step: 1 ___________________________________________
val_loss: 165.61190795898438


Step: 2 ___________________________________________
val_loss: 141.94589233398438


Step: 3 ___________________________________________
val_loss: 176.6719970703125


Step: 4 ___________________________________________
val_loss: 119.45951843261719
winner_seed: 797226887


Step: 5 ___________________________________________
val_loss: 111.49437713623047
winner_seed: 1847975421


Step: 6 ___________________________________________
val_loss: 152.91616821289062


Step: 7 ___________________________________________

In [43]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 24ms/step - loss: 21834.6875 - val_loss: 16775.8047
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 12780.9297 - val_loss: 15053.5010
Epoch 3/10000
5/5 [==============================] - 0s 6ms/step - loss: 15036.9062 - val_loss: 6653.5327
Epoch 4/10000
5/5 [==============================] - 0s 5ms/step - loss: 2703.4895 - val_loss: 1189.1489
Epoch 5/10000
5/5 [==============================] - 0s 6ms/step - loss: 978.4602 - val_loss: 977.1393
Epoch 6/10000
5/5 [==============================] - 0s 6ms/step - loss: 913.7336 - val_loss: 713.6136
Epoch 7/10000
5/5 [==============================] - 0s 5ms/step - loss: 724.8107 - val_loss: 531.9487
Epoch 8/10000
5/5 [==============================] - 0s 5ms/step - loss: 770.2656 - val_loss: 631.6113
Epoch 9/10000
5/5 [==============================] - 0s 5ms/step - loss: 769.2707 - val_loss: 645.3582
Epoch 10/10000
5/5 [==============================] - 0s 5m

5/5 [==============================] - 0s 5ms/step - loss: 411.6803 - val_loss: 419.0473
Epoch 80/10000
5/5 [==============================] - 0s 6ms/step - loss: 395.6865 - val_loss: 344.3215
Epoch 81/10000
5/5 [==============================] - 0s 6ms/step - loss: 346.4088 - val_loss: 322.7509
Epoch 82/10000
5/5 [==============================] - 0s 5ms/step - loss: 329.1820 - val_loss: 407.4732
Epoch 83/10000
5/5 [==============================] - 0s 5ms/step - loss: 359.2796 - val_loss: 319.1651
Epoch 84/10000
5/5 [==============================] - 0s 5ms/step - loss: 417.3120 - val_loss: 295.0878
Epoch 85/10000
5/5 [==============================] - 0s 5ms/step - loss: 481.7678 - val_loss: 415.9739
Epoch 86/10000
5/5 [==============================] - 0s 5ms/step - loss: 367.0396 - val_loss: 784.2083
Epoch 87/10000
5/5 [==============================] - 0s 5ms/step - loss: 508.0570 - val_loss: 540.8310
Epoch 88/10000
5/5 [==============================] - 0s 6ms/step - loss: 384.4

5/5 [==============================] - 0s 6ms/step - loss: 631.7002 - val_loss: 518.1969
Epoch 158/10000
5/5 [==============================] - 0s 5ms/step - loss: 612.2380 - val_loss: 467.7152
Epoch 159/10000
5/5 [==============================] - 0s 5ms/step - loss: 570.3353 - val_loss: 597.1425
Epoch 160/10000
5/5 [==============================] - 0s 5ms/step - loss: 700.2546 - val_loss: 510.9258
Epoch 161/10000
5/5 [==============================] - 0s 5ms/step - loss: 732.6822 - val_loss: 515.3074
Epoch 162/10000
5/5 [==============================] - 0s 5ms/step - loss: 685.9822 - val_loss: 721.5646
Epoch 163/10000
5/5 [==============================] - 0s 5ms/step - loss: 665.1136 - val_loss: 460.7883
Epoch 164/10000
5/5 [==============================] - 0s 5ms/step - loss: 690.9684 - val_loss: 500.1165
Epoch 165/10000
5/5 [==============================] - 0s 5ms/step - loss: 599.3869 - val_loss: 355.9166
Epoch 166/10000
5/5 [==============================] - 0s 5ms/step - lo

5/5 [==============================] - 0s 6ms/step - loss: 381.7684 - val_loss: 386.3570
Epoch 236/10000
5/5 [==============================] - 0s 6ms/step - loss: 359.8732 - val_loss: 283.7314
Epoch 237/10000
5/5 [==============================] - 0s 5ms/step - loss: 363.5246 - val_loss: 320.7913
Epoch 238/10000
5/5 [==============================] - 0s 5ms/step - loss: 351.2713 - val_loss: 480.5967
Epoch 239/10000
5/5 [==============================] - 0s 6ms/step - loss: 392.2897 - val_loss: 388.2650
Epoch 240/10000
5/5 [==============================] - 0s 5ms/step - loss: 380.7838 - val_loss: 310.8898
Epoch 241/10000
5/5 [==============================] - 0s 6ms/step - loss: 479.0691 - val_loss: 320.0591
Epoch 242/10000
5/5 [==============================] - 0s 5ms/step - loss: 428.2172 - val_loss: 332.0376
Epoch 243/10000
5/5 [==============================] - 0s 6ms/step - loss: 381.9767 - val_loss: 409.0473
Epoch 244/10000
5/5 [==============================] - 0s 6ms/step - lo

5/5 [==============================] - 0s 5ms/step - loss: 486.3920 - val_loss: 496.4784
Epoch 314/10000
5/5 [==============================] - 0s 6ms/step - loss: 398.5899 - val_loss: 434.2252
Epoch 315/10000
5/5 [==============================] - 0s 6ms/step - loss: 530.1013 - val_loss: 330.5523
Epoch 316/10000
5/5 [==============================] - 0s 6ms/step - loss: 338.0728 - val_loss: 634.6502
Epoch 317/10000
5/5 [==============================] - 0s 6ms/step - loss: 414.3628 - val_loss: 312.8499
Epoch 318/10000
5/5 [==============================] - 0s 5ms/step - loss: 333.6009 - val_loss: 351.7093
Epoch 319/10000
5/5 [==============================] - 0s 5ms/step - loss: 346.8552 - val_loss: 432.8091
Epoch 320/10000
5/5 [==============================] - 0s 6ms/step - loss: 459.5528 - val_loss: 401.9594
Epoch 321/10000
5/5 [==============================] - 0s 6ms/step - loss: 476.5906 - val_loss: 338.8000
Epoch 322/10000
5/5 [==============================] - 0s 5ms/step - lo

...

5/5 [==============================] - 0s 5ms/step - loss: 331.4408 - val_loss: 298.8382
Epoch 1170/10000
5/5 [==============================] - 0s 5ms/step - loss: 318.8876 - val_loss: 360.9723
Epoch 1171/10000
5/5 [==============================] - 0s 5ms/step - loss: 300.4687 - val_loss: 295.0268
Epoch 1172/10000
5/5 [==============================] - 0s 6ms/step - loss: 332.3128 - val_loss: 541.3492
Epoch 1173/10000
5/5 [==============================] - 0s 5ms/step - loss: 401.3989 - val_loss: 422.3801
Epoch 1174/10000
5/5 [==============================] - 0s 5ms/step - loss: 307.6537 - val_loss: 250.7718
Epoch 1175/10000
5/5 [==============================] - 0s 5ms/step - loss: 326.4710 - val_loss: 246.8161
Epoch 1176/10000
5/5 [==============================] - 0s 5ms/step - loss: 293.5417 - val_loss: 114.2801
Epoch 1177/10000
5/5 [==============================] - 0s 5ms/step - loss: 310.2494 - val_loss: 173.3269
Epoch 1178/10000
5/5 [==============================] - 0s 5ms/

5/5 [==============================] - 0s 5ms/step - loss: 382.0836 - val_loss: 227.3324
Epoch 1247/10000
5/5 [==============================] - 0s 5ms/step - loss: 337.8766 - val_loss: 215.1450
Epoch 1248/10000
5/5 [==============================] - 0s 5ms/step - loss: 371.8792 - val_loss: 211.7922
Epoch 1249/10000
5/5 [==============================] - 0s 5ms/step - loss: 380.3015 - val_loss: 155.0340
Epoch 1250/10000
5/5 [==============================] - 0s 5ms/step - loss: 341.5769 - val_loss: 147.8850
Epoch 1251/10000
5/5 [==============================] - 0s 5ms/step - loss: 336.9479 - val_loss: 275.3700
Epoch 1252/10000
5/5 [==============================] - 0s 5ms/step - loss: 389.4438 - val_loss: 399.0834
Epoch 1253/10000
5/5 [==============================] - 0s 5ms/step - loss: 420.3351 - val_loss: 272.6197
Epoch 1254/10000
5/5 [==============================] - 0s 5ms/step - loss: 392.5325 - val_loss: 174.0527
Epoch 1255/10000
5/5 [==============================] - 0s 6ms/

5/5 [==============================] - 0s 5ms/step - loss: 357.2881 - val_loss: 408.7736
Epoch 1324/10000
5/5 [==============================] - 0s 6ms/step - loss: 384.3383 - val_loss: 278.7546
Epoch 1325/10000
5/5 [==============================] - 0s 5ms/step - loss: 282.2574 - val_loss: 256.8210
Epoch 1326/10000
5/5 [==============================] - 0s 5ms/step - loss: 319.2173 - val_loss: 324.8228
Epoch 1327/10000
5/5 [==============================] - 0s 5ms/step - loss: 364.9015 - val_loss: 314.8830
Epoch 1328/10000
5/5 [==============================] - 0s 6ms/step - loss: 345.7065 - val_loss: 211.1769
Epoch 1329/10000
5/5 [==============================] - 0s 6ms/step - loss: 355.6053 - val_loss: 258.6931
Epoch 1330/10000
5/5 [==============================] - 0s 6ms/step - loss: 355.5418 - val_loss: 346.9986
Epoch 1331/10000
5/5 [==============================] - 0s 5ms/step - loss: 405.8615 - val_loss: 255.2562
Epoch 1332/10000
5/5 [==============================] - 0s 6ms/

5/5 [==============================] - 0s 6ms/step - loss: 307.3673 - val_loss: 259.7151
Epoch 1401/10000
5/5 [==============================] - 0s 6ms/step - loss: 281.9341 - val_loss: 233.5352
Epoch 1402/10000
5/5 [==============================] - 0s 5ms/step - loss: 287.3471 - val_loss: 276.0061
Epoch 1403/10000
5/5 [==============================] - 0s 5ms/step - loss: 313.4730 - val_loss: 259.1403
Epoch 1404/10000
5/5 [==============================] - 0s 5ms/step - loss: 350.8009 - val_loss: 330.4230
Epoch 1405/10000
5/5 [==============================] - 0s 5ms/step - loss: 351.0956 - val_loss: 355.2036
Epoch 1406/10000
5/5 [==============================] - 0s 5ms/step - loss: 342.2296 - val_loss: 248.7325
Epoch 1407/10000
5/5 [==============================] - 0s 5ms/step - loss: 288.4031 - val_loss: 267.0314
Epoch 1408/10000
5/5 [==============================] - 0s 5ms/step - loss: 306.3088 - val_loss: 210.7468
Epoch 1409/10000
5/5 [==============================] - 0s 6ms/

5/5 [==============================] - 0s 5ms/step - loss: 281.4973 - val_loss: 344.7981
Epoch 1478/10000
5/5 [==============================] - 0s 5ms/step - loss: 294.3922 - val_loss: 300.2815
Epoch 1479/10000
5/5 [==============================] - 0s 5ms/step - loss: 318.5048 - val_loss: 296.1721
Epoch 1480/10000
5/5 [==============================] - 0s 8ms/step - loss: 348.9326 - val_loss: 307.3746
Epoch 1480: early stopping


In [44]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [45]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 16ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,179.093781,179.093781,179.09375,179.09375,179.093689,179.093506,176.744965,176.983459,177.061829,176.679138,171.590363,179.090179,167.704407,168.262878,173.095001,176.321564,178.01825,185.202789,175.1185,187.085693,197.962799,27.509636,17.151037,-10.539238,-7.702671,0.599251,21.364222,145.607819,196.003052,167.77063
Target,179.243,194.134,186.948,194.907,179.505,173.278,160.261,113.987,114.289,117.313,110.502,147.996,152.386,177.259,155.913,172.41,179.419,157.165,151.601,129.537,105.236,110.835,127.925,127.138,157.128,164.44,140.298,168.407,163.615,155.07
Error,0.149216,15.040222,7.854248,15.813248,0.411316,5.815506,16.483963,62.99646,62.772827,59.366135,61.088364,31.094177,15.318405,8.996124,17.182007,3.91156,1.400757,28.037796,23.517502,57.548691,92.726799,83.325363,110.773964,137.677246,164.830673,163.840759,118.933784,22.799179,32.388046,12.700623


In [46]:
display(mae)
display(mape)

47.826504

0.35569668

In [47]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [48]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[2132.7122]] - Target[1872.363]| =  Error: [[260.34912]]; MAPE:[[0.13904841]]
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[1642.8933]] - Target[1746.8239999999998]| =  Error: [[103.930664]]; MAPE:[[0.05949693]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[523.64233]] - Target[948.9580000000001]| =  Error: [[425.31567]]; MAPE:[[0.4481923]]


[array([[260.34912]], dtype=float32),
 array([[103.930664]], dtype=float32),
 array([[425.31567]], dtype=float32)]

263.1985

0.21557921